# Chain of Density
- 요약의 품질을 높이려면 포괄적이면서도 엔티티 중심이어야 한다.
- CoD는 요약을 위한 Prompt 기법의 일종이며, 최대한 정보 손실을 줄이고 주요 Entity가 잘 잡히도록 한다.
- CoD의 초기 단계에는 제한된 엔티티만을 나열하지만 이후 단계에서는 점차 확장되어 누락된 엔티티나 정보를 추가한다.
## 참고링크
- https://jiniai.biz/?p=1694

In [2]:
# API 키를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API 키 정보 로드
load_dotenv()

True

In [3]:
from langchain import hub
from langchain_core.output_parsers import JsonOutputParser
from langchain.document_loaders import WebBaseLoader
from langchain.callbacks.base import BaseCallbackHandler
from langchain_community.chat_models import ChatOllama
from langchain_core.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

prompt = hub.pull("teddynote/chain-of-density-korean")
prompt


ChatPromptTemplate(input_variables=['ARTICLE'], metadata={'lc_hub_owner': 'teddynote', 'lc_hub_repo': 'chain-of-density-korean', 'lc_hub_commit_hash': 'f7122cae5cd37900b29543353cb15dbd8dd8f0a6fd20c4413a9a958c19b800c9'}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['ARTICLE'], template='Article: {ARTICLE}\nYou will generate increasingly concise, entity-dense summaries of the above article. \n\nRepeat the following 2 steps 5 times. \n\nStep 1. Identify 1-3 informative entities (";" delimited) from the article which are missing from the previously generated summary. \nStep 2. Write a new, denser summary of identical length which covers every entity and detail from the previous summary plus the missing entities. \n\nA missing entity is:\n- relevant to the main story, \n- specific yet concise (100 words or fewer), \n- novel (not in the previous summary), \n- faithful (present in the article), \n- anywhere (can be located anywhere in the article).\n\nGuidelines

In [9]:
loader = WebBaseLoader("https://www.aitimes.com/news/articleView.html?idxno=131777")
docs = loader.load()
content = docs[0].page_content

callbacks = [StreamingStdOutCallbackHandler()]
llm = ChatOllama(model="gemma:7b", temperature=0, streaming=True, callbacks=callbacks)

# Create the chain, including
chain = (
    prompt
    | llm
    | JsonOutputParser()
    | (lambda x: x[-1]["Denser_Summary"])
)

# Invoke the chain
result = chain.invoke({"ARTICLE": content})
print(result)

[
  {
    "Missing_Entities": "AI 교육, AI융합연구센터",
    "Denser_Summary": "이 기사는 AI 교육의 필요성과 AI융합연구센터의 활동에 대해 다룬다. 또한, AI 대학원과 관련된 정보도 포함된다."
  },
  {
    "Missing_Entities": "칼럼, 기고, 기자수첩",
    "Denser_Summary": "오피니언 섹션은 칼럼, 기고, 기자수첩과 같은 다양한 콘텐츠를 제공하고 있다. 특히, 박정현의 데이터사이언스와 조영임의 AI단상이 주목할 만하다."
  },
  {
    "Missing_Entities": "CES 2023, AI & BIG DATA SHOW",
    "Denser_Summary": "이벤트 섹션은 CES 2023과 AI & BIG DATA SHOW와 같은 향후 이벤트에 대한 정보를 제공한다."
  },
  {
    "Missing_Entities": "춤추는 AI개발자 연봉, 로봇이 온다",
    "Denser_Summary": "
"

